In [ ]:
!pip install tensorflow==1.13.1
!pip install keras==2.2.5
!pip install mrcnn
!pip install scikit-image
!pip install imgaug
!pip install IPython
!pip install 'h5py==2.10.0' --force-reinstall

In [ ]:
from skimage.io import imread
import matplotlib.pyplot as plt
import mrcnn.utils as utils
import mrcnn.model as modellib
import numpy as np
import pandas as pd
import tensorflow as tf

import os
import sys
import json
import time
import skimage
import imageio
import glob
import imgaug
import multiprocessing

from PIL import Image, ImageDraw
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold
from PIL import Image, ImageEnhance
from mrcnn.config import Config
from mrcnn import visualize

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Preparing Dataset

In [ ]:
HEIGHT = 520
WIDTH = 704
SHAPE = (HEIGHT, WIDTH)

In [ ]:
traindf = pd.read_csv('../input/sartorius-cell-instance-segmentation'+'/train.csv')

id_unique = traindf['id'].unique()

def get_file_path(image_id):
    return f'/kaggle/input/sartorius-cell-instance-segmentation/train/{image_id}.png'
traindf['file_path'] = traindf['id'].apply(get_file_path)

CELL_NAMES = np.sort(traindf['cell_type'].unique())
CELL_NAMES_DICT = dict([(v, k) for k, v in enumerate(CELL_NAMES)])

traindf['cell_type_label'] = traindf['cell_type'].apply(CELL_NAMES_DICT.get) + 1
ID2CELL_LABEL = dict(
    [(k, v) for k, v in traindf[['id', 'cell_type_label']].itertuples(name=None, index=False)]
)

traindf.head()

In [ ]:
def rle_decode(annotations, shape=(520, 704)):
    '''
    returns binary masked map 
    '''
    s = annotations.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

In [ ]:
id = "0030fd0e6378"
img = imread("../input/sartorius-cell-instance-segmentation/train/0030fd0e6378.png")
plt.imshow(img, cmap='gray')

In [ ]:
imageMaks=traindf[traindf['id']==id]
allMask = np.zeros(img.shape)
for i in range(len(imageMaks['annotation'])):
    allMask = allMask + rle_decode(imageMaks['annotation'].iloc[i])
plt.imshow(allMask, cmap="Greens")

In [ ]:
def rle_decode_by_image_id(image_id):
    rows = traindf.loc[traindf['id'] == image_id]
    
    # Image Shape
    mask = np.full(shape=[len(rows), np.prod(SHAPE)], fill_value=0, dtype=np.uint8)
    
    for idx, (_, row) in enumerate(rows.iterrows()):
        s = row['annotation'].split()
        starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
        starts -= 1
        ends = starts + lengths
        for lo, hi in zip(starts, ends):
            mask[idx, lo:hi] = True
    
    mask = mask.reshape([len(rows), *SHAPE])
    mask = np.moveaxis(mask, 0, 2)
    
    return mask

In [ ]:
!rm -rf ./train ./test
!mkdir ./train ./test

In [ ]:
class CellDataset(utils.Dataset):

    def load_data(self, image_ids, form, image_group):
   
        for i, name in enumerate(CELL_NAMES):
            self.add_class('cell', 1 + i, name)
       
        # Add the image using the base method from utils.Dataset
        for vertical_flip in [True, False]:
            for horizontal_flip in [True, False]:
                for image in tqdm(image_ids):
                    self.add_image('cell', 
                           image_id=image, 
                           path=(f'../input/sartorius-cell-instance-segmentation/train/{image}.png'),
                           label = ID2CELL_LABEL[image],
                           height=512, width=512,
                          vertical_flip=vertical_flip, horizontal_flip=horizontal_flip,
                      )
            
            
    def load_mask(self, image_id):
        """ Load instance masks for the given image.
        MaskRCNN expects masks in the form of a bitmap [height, width, instances].
        Args:
            image_id: The id of the image to load masks for
        Returns:
            masks: A bool array of shape [height, width, instance count] with
                one mask per instance.
            class_ids: a 1D array of class IDs of the instance masks.
        """
    
        info = self.image_info[image_id]
        image_id = info['id']
    
        # Get masks by image_id
        masks = rle_decode_by_image_id(image_id)
        #masks = pad_image(masks, 0)

        # Get label
        _, _, size = masks.shape
        label = info['label']
        class_ids = np.full(size, label, dtype=np.int32)
        
        return masks, class_ids

In [ ]:
np.random.shuffle(id_unique)
id_unique_test, id_unique_val = id_unique[:600], id_unique[600:]

In [ ]:
dataset_train = CellDataset()
dataset_train.load_data(id_unique_test, 'png', 'train')
dataset_train.prepare()
dataset_val = CellDataset()
dataset_val.load_data(id_unique_val, 'png', 'validation')
dataset_val.prepare()

# Modelling

In [ ]:
BATCH_SIZE = 1
N_SAMPLES = traindf['id'].nunique()
class CellConfig(Config):
    """Configuration for training on the cigarette butts dataset.
    Derives from the base Config class and overrides values specific
    to the cigarette butts dataset.
    """
    
    NAME = "cell"

    # Set batch size to 1.
    GPU_COUNT = 1
    IMAGES_PER_GPU = BATCH_SIZE
    STEPS_PER_EPOCH = int(N_SAMPLES / BATCH_SIZE)
    
    # Number of Classes
    NUM_CLASSES = 1 + len(CELL_NAMES)

    # Input image resizing
    # Random crops of size 512x512
    IMAGE_RESIZE_MODE = "crop"
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    IMAGE_MIN_SCALE = 2.0

    
    BACKBONE = "resnet50"

    # Training Structure
    FPN_CLASSIF_FC_LAYERS_SIZE = 1024
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    # Regions of Interest
    PRE_NMS_LIMIT = 6000
    # Non Max Supression
    POST_NMS_ROIS_TRAINING = 2000
    POST_NMS_ROIS_INFERENCE = 2000
    # Instances
    MAX_GT_INSTANCES = 790
    TRAIN_ROIS_PER_IMAGE = 200
    DETECTION_MAX_INSTANCES = 200
    
    # Thresholds
    RPN_NMS_THRESHOLD = 0.70        # IoU Threshold for RPN proposals and GT
    DETECTION_MIN_CONFIDENCE = 0.50 # Non-Background Confidence Threshold
    DETECTION_NMS_THRESHOLD = 0.30  # IoU Threshold for ROI and GT
    ROI_POSITIVE_RATIO = 0.33
    
    # Prediction Mask Shape
    USE_MINI_MASK = True
    MINI_MASK_SHAPE = (56, 56)
    
    # DO NOT train Batch Normalization because of small batch size
    # There are too few samples to correctly train the normalization
    TRAIN_BN = False
    
    # Learning Rate
    LEARNING_RATE = 0.01
    WEIGHT_DECAY = 0.0
    N_WARMUP_STEPS = 2
    LR_SCHEDULE = True
    
    # Dataloader Queue Size (was set to 100 but resulted in OOM error)
    MAX_QUEUE_SIZE = 10
    
    # Cache Items
    CACHE = True
    
    # Debug mode will disable model checkpoints
    DEBUG = False
    
    # Do not use multithreading as this slows down the dataloader!
    WORKERS = 0
    
    # Losses
    LOSS_WEIGHTS = {
        'rpn_class_loss': 1.0,    # is the class of the bbox correct? / RPN anchor classifier loss (Forground/Background)
        'rpn_bbox_loss': 1.0,     # is the size of the bbox correct? / RPN bounding box loss graph (bbox of generic object)
        'mrcnn_class_loss': 1.0,  # loss for the classifier head of Mask R-CNN (Background / specific class)
        'mrcnn_bbox_loss': 1.0,   # is the size of the bounding box correct or not? / loss for Mask R-CNN bounding box refinement
        'mrcnn_mask_loss': 1.0,   # is the class correct? is the pixel correctly assign to the class? / mask binary cross-entropy loss for the masks head
    }
    
config = CellConfig()
config.display()

In [ ]:
model = modellib.MaskRCNN(mode="training", config=config, model_dir='model_checkpoints')

In [ ]:
# path to COCO-dataset weights
COCO_MODEL_PATH = './mask_rcnn_coco.h5'

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
exclude = ["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"]
    
# Excluce FC layer if it is not the original size
if config.FPN_CLASSIF_FC_LAYERS_SIZE != 1024:
    print(f'Excluding FC layer')
    exclude += [
        "mrcnn_class_conv1", "mrcnn_class_bn1", "mrcnn_class_conv2", "mrcnn_class_bn2",
    ]
    
# using coco weights
model.load_weights(
    COCO_MODEL_PATH,
    by_name=True,
    exclude=exclude,
)

In [ ]:
history = model.train(
    dataset_train, dataset_val, 
    learning_rate=config.LEARNING_RATE,
    epochs=3, 
    layers="all",
)

In [ ]:
model.keras_model.save("./maskrcnn.h5")